# 3-Stage Clos Network with Port Channels
###### <sup>Inputs:  Bhavani Parise, Deepti Chandra; Developed by Sarah Samuel


The [3 Stage Clos Notebook](./3StageClos-Notebook.ipynb) walked you through the step-by-step procedure to configure a 3 stage Clos network on Cisco 8000 routers that run SONiC operating system. This notebook shows you how to configure a 3-stage Clos network with port channels.

The 3-stage Clos network is a robust IP-BGP underlay network that enables the servers to communicate with each other with minimum latency.

A port channel is a virtual interface on a network node such as a router or a switch that aggregate multiple physical interfaces into one logical interface.

When port channels are used instead of physical interfaces in a 3-stage Clos network, the network has the following benefits:
* Increased aggregate bandwidth of a link by distributing traffic among all functional members in the port channel.
* Since the port channel load balances the traffic across multiple member links, there is optimum bandwidth usage.
* Link redundancy - In case one member link fails, traffic previously carried on this link is switched to the remaining member links. If a member link in a port channel goes down, the upper protocols are not aware of it. Just the bandwidth is reduced. The MAC address tables are not affected by link failure either.

The following topology diagram depicts a simple 3-stage Clos network with port channels:

<center><img src="images/522667.jpg" width="700"/></center>


As you read through this notebook, play the code-cells, using the play button in the top left corner of this page, to send configuration commands to the live network nodes on the Cisco 8000 Emulator that runs in the background. The notebook refreshes the output of each cell-execution just beneath it.

[Access Device Consoles](#Access-Device-Consoles) and then play through the following steps to [Bring Up 3 Stage Clos Network with Port Channels](#Bring-Up-3-Stage-Clos-Network-with-Port-Channels):
* [Configure Host-Names](#Configure-Host-Names)
* [Set up Port Channels](#Set-up-Port-Channels)
* [Assign IP-Addresses](#Assign-IP-Addresses)
* [Configure eBGP](#Configure-eBGP)
* [Verify BGP Route Exchange](#Verify-BGP-Route-Exchange)
* [Send Traffic from TREX](#Send-Traffic-from-TREX)
* [Verify Traffic Statistics](#Verify-Traffic-Statistics)

Finally, [clean up emulator session](#Clean-Up-Emulator-Session), once you are done.

### Access Device Consoles

> Install prerequisite modules: This is required mainly for the first time you play this notebook. You can skip playing this cell for the subsequent runs.

In [1]:
%%capture cell-output
!sh lib/prereq_install.sh

> Play the following cell to bring up topology and access ssh console of each device in the topology. This cell takes about 10 - 15 minutes to complete execution. When you see the **Sim status** displayed as  **{'localhost': 'running'}** underneath this cell, the topology is up. Wait for a few more seconds until the code accesses the ssh console of each device. 

> Avoid playing this cell more than once, without [cleaning up the emulator session](#Clean-Up-Emulator-Session).

In [2]:
from lib.leaf_spine import *
nodes = {
         'S0':'', 
         'S1':'',
         'L0':'', 
         'L1':'', 
         'trex':''
        }

tb = access_device_consoles("lib/leaf_spine.yaml", nodes)

3.6.9 (default, Jun 29 2022, 11:45:57) 
[GCC 8.4.0]

*** Logging into the devices ***

2022-09-01 17:45:33,859: %UNICON-INFO: +++ S0 logfile /tmp/S0-cli-20220901T174533858.log +++

2022-09-01 17:45:33,860: %UNICON-INFO: +++ Unicon plugin linux (unicon.plugins.linux) +++

2022-09-01 17:45:33,867: %UNICON-INFO: +++ connection to spawn: ssh -l cisco 10.10.20.228 -p 60003, id: 140516160311024 +++

2022-09-01 17:45:33,871: %UNICON-INFO: connection to S0
The authenticity of host '[10.10.20.228]:60003 ([10.10.20.228]:60003)' can't be established.
ECDSA key fingerprint is SHA256:BYsuXxg4AWOcokcLrkXUgzxbHBLv722mvYj+b03f2AE.
Are you sure you want to continue connecting (yes/no)? yes
cisco@10.10.20.228's password: 
Linux sonic 4.19.0-12-2-amd64 #1 SMP Debian 4.19.152-1 (2020-10-18) x86_64
You are on
  ____   ___  _   _ _  ____
 / ___| / _ \| \ | (_)/ ___|
 \___ \| | | |  \| | | |
  ___) | |_| | |\  | | |___
 |____/ \___/|_| \_|_|\____|

-- Software for Open Networking in the Cloud --

Unauthorize

## Bring Up 3-Stage Clos Network with Port Channels

> To configure a 3-stage Clos network, continue playing through the following steps:

### Configure Host-Names

> Configure host names for the spine and leaf routers for easy identification.

In [25]:
out = nodes['S0'].execute('sudo config hostname SPINE0')
out = nodes['S0'].execute('sudo config save -y')
out = nodes['S1'].execute('sudo config hostname SPINE1')
out = nodes['S1'].execute('sudo config save -y')
out = nodes['L0'].execute('sudo config hostname LEAF0')
out = nodes['L0'].execute('sudo config save -y')
out = nodes['L1'].execute('sudo config hostname LEAF1')
out = nodes['L1'].execute('sudo config save -y')


2022-09-01 17:59:22,912: %UNICON-INFO: +++ sonic with via 'cli': executing command 'sudo config hostname SPINE0' +++
sudo config hostname SPINE0
Running command: service hostname-config restart
Reloading Monit configuration ...
Reinitializing monit daemon
Please note loaded setting will be lost after system reboot. To preserve setting, run `config save`.
cisco@sonic:~$ 

2022-09-01 17:59:24,042: %UNICON-INFO: +++ sonic with via 'cli': executing command 'sudo config save -y' +++
sudo config save -y
Running command: /usr/local/bin/sonic-cfggen -d --print-data > /etc/sonic/config_db.json
cisco@sonic:~$ 

2022-09-01 17:59:24,914: %UNICON-INFO: +++ sonic with via 'cli': executing command 'sudo config hostname SPINE1' +++
sudo config hostname SPINE1
Running command: service hostname-config restart
Reloading Monit configuration ...
Reinitializing monit daemon
Please note loaded setting will be lost after system reboot. To preserve setting, run `config save`.
cisco@sonic:~$ 

2022-09-01 17:59

### Set up Port Channels
> In this section, we will see how to configure regular port channels. Optionally, you can also
* [Set up Port Channels with Min-Links](#Set-up-Port-Channels-with-Min-Links)
* [Set up Port Channels with Fallback Option](#Set-up-Port-Channels-with-Fallback-Option)

> Configure port channels and add members to the port channel.

In [26]:
out = nodes['S0'].execute('sudo config portchannel add PortChannel1')
out = nodes['S0'].execute('sudo config portchannel member add PortChannel1 Ethernet0')
out = nodes['S0'].execute('sudo config portchannel member add PortChannel1 Ethernet1')
out = nodes['S0'].execute('sudo config portchannel add PortChannel2')
out = nodes['S0'].execute('sudo config portchannel member add PortChannel2 Ethernet2')
out = nodes['S0'].execute('sudo config portchannel member add PortChannel2 Ethernet3')
out = nodes['S1'].execute('sudo config portchannel add PortChannel1')
out = nodes['S1'].execute('sudo config portchannel member add PortChannel1 Ethernet0')
out = nodes['S1'].execute('sudo config portchannel member add PortChannel1 Ethernet1')
out = nodes['S1'].execute('sudo config portchannel add PortChannel2')
out = nodes['S1'].execute('sudo config portchannel member add PortChannel2 Ethernet2')
out = nodes['S1'].execute('sudo config portchannel member add PortChannel2 Ethernet3')
out = nodes['L0'].execute('sudo config portchannel add PortChannel1')
out = nodes['L0'].execute('sudo config portchannel member add PortChannel1 Ethernet0')
out = nodes['L0'].execute('sudo config portchannel member add PortChannel1 Ethernet1')
out = nodes['L0'].execute('sudo config portchannel add PortChannel2')
out = nodes['L0'].execute('sudo config portchannel member add PortChannel2 Ethernet3')
out = nodes['L0'].execute('sudo config portchannel member add PortChannel2 Ethernet4')
out = nodes['L1'].execute('sudo config portchannel add PortChannel1')
out = nodes['L1'].execute('sudo config portchannel member add PortChannel1 Ethernet0')
out = nodes['L1'].execute('sudo config portchannel member add PortChannel1 Ethernet1')
out = nodes['L1'].execute('sudo config portchannel add PortChannel2')
out = nodes['L1'].execute('sudo config portchannel member add PortChannel2 Ethernet3')
out = nodes['L1'].execute('sudo config portchannel member add PortChannel2 Ethernet4')


2022-09-01 18:00:00,456: %UNICON-INFO: +++ sonic with via 'cli': executing command 'sudo config portchannel add PortChannel1' +++
sudo config portchannel add PortChannel1
cisco@sonic:~$ 

2022-09-01 18:00:01,094: %UNICON-INFO: +++ sonic with via 'cli': executing command 'sudo config portchannel member add PortChannel1 Ethernet0' +++
sudo config portchannel member add PortChannel1 Ethernet0
cisco@sonic:~$ 

2022-09-01 18:00:01,755: %UNICON-INFO: +++ sonic with via 'cli': executing command 'sudo config portchannel member add PortChannel1 Ethernet1' +++
sudo config portchannel member add PortChannel1 Ethernet1
cisco@sonic:~$ 

2022-09-01 18:00:02,419: %UNICON-INFO: +++ sonic with via 'cli': executing command 'sudo config portchannel add PortChannel2' +++
sudo config portchannel add PortChannel2
cisco@sonic:~$ 

2022-09-01 18:00:02,988: %UNICON-INFO: +++ sonic with via 'cli': executing command 'sudo config portchannel member add PortChannel2 Ethernet2' +++
sudo config portchannel member a

> Verify the port channel configurations on all routers.

In [27]:
for n in nodes:
   if (n != 'trex'):
      out = nodes[n].execute('show interfaces portchannel')
      out = nodes[n].execute('show interface status Ethernet0-4')   
      out = nodes[n].execute('show interfaces status PortChannel1-2')


2022-09-01 18:00:40,391: %UNICON-INFO: +++ sonic with via 'cli': executing command 'show interfaces portchannel' +++
show interfaces portchannel
Flags: A - active, I - inactive, Up - up, Dw - Down, N/A - not available,
       S - selected, D - deselected, * - not synced
  No.  Team Dev      Protocol     Ports
-----  ------------  -----------  -------------------------
    1  PortChannel1  LACP(A)(Up)  Ethernet1(S) Ethernet0(S)
    2  PortChannel2  LACP(A)(Up)  Ethernet2(S) Ethernet3(S)
cisco@sonic:~$ 

2022-09-01 18:00:40,973: %UNICON-INFO: +++ sonic with via 'cli': executing command 'show interface status Ethernet0-4' +++
show interface status Ethernet0-4
  Interface                Lanes    Speed    MTU    FEC      Alias          Vlan    Oper    Admin             Type    Asym PFC
-----------  -------------------  -------  -----  -----  ---------  ------------  ------  -------  ---------------  ----------
  Ethernet0  2304,2305,2306,2307     100G   9100    N/A  Ethernet0  PortChannel1

#### Set up Port Channels with Min-Links
> To ensure the port channel does not remain **Up**, if minimum number of member-links are not **Up**, configure the **min-links** option while configuring the port channel. This is an optional configuration. In the below cell, we will configure PortChannel1 of SPINE0 router with **min-links** of 2. 
> First, delete the existing port channel configurations and then reconfigure with min-links option.

In [28]:
# Deleting the existing port channel configurations 
out = nodes['S0'].execute('sudo config portchannel member del PortChannel1 Ethernet0')
out = nodes['S0'].execute('sudo config portchannel member del PortChannel1 Ethernet1')
out = nodes['S0'].execute('sudo config portchannel del PortChannel1')

# Configure the port channel with min-links of 2
out = nodes['S0'].execute('sudo config portchannel add PortChannel1 --min-links 2')
out = nodes['S0'].execute('sudo config portchannel member add PortChannel1 Ethernet0')
out = nodes['S0'].execute('sudo config portchannel member add PortChannel1 Ethernet1')

# Verify the port channel 
out = nodes['S0'].execute('show interface status Ethernet0-1')   
out = nodes['S0'].execute('show interfaces status PortChannel1')


2022-09-01 18:00:53,337: %UNICON-INFO: +++ sonic with via 'cli': executing command 'sudo config portchannel member del PortChannel1 Ethernet0' +++
sudo config portchannel member del PortChannel1 Ethernet0
cisco@sonic:~$ 

2022-09-01 18:00:53,935: %UNICON-INFO: +++ sonic with via 'cli': executing command 'sudo config portchannel member del PortChannel1 Ethernet1' +++
sudo config portchannel member del PortChannel1 Ethernet1
cisco@sonic:~$ 

2022-09-01 18:00:54,570: %UNICON-INFO: +++ sonic with via 'cli': executing command 'sudo config portchannel del PortChannel1' +++
sudo config portchannel del PortChannel1
cisco@sonic:~$ 

2022-09-01 18:00:55,126: %UNICON-INFO: +++ sonic with via 'cli': executing command 'sudo config portchannel add PortChannel1 --min-links 2' +++
sudo config portchannel add PortChannel1 --min-links 2
cisco@sonic:~$ 

2022-09-01 18:00:55,722: %UNICON-INFO: +++ sonic with via 'cli': executing command 'sudo config portchannel member add PortChannel1 Ethernet0' +++
sudo

> We can verify if min-links works as expected. If we shutdown one of the member-links of PortChannel1 i.e. Ethernet0, the **Oper** state of the port channel should be **down** in the output of ```show interfaces status PortChannel1``` 

In [29]:
out = nodes['S0'].execute('sudo config interface shutdown Ethernet0')

# Verify that port channel is down when minimum number of member links (ie 2) are not Up
out = nodes['S0'].execute('show interface status Ethernet0-1')   
out = nodes['S0'].execute('show interfaces status PortChannel1')


2022-09-01 18:03:26,664: %UNICON-INFO: +++ sonic with via 'cli': executing command 'sudo config interface shutdown Ethernet0' +++
sudo config interface shutdown Ethernet0
cisco@sonic:~$ 

2022-09-01 18:03:27,238: %UNICON-INFO: +++ sonic with via 'cli': executing command 'show interface status Ethernet0-1' +++
show interface status Ethernet0-1
  Interface                Lanes    Speed    MTU    FEC      Alias          Vlan    Oper    Admin             Type    Asym PFC
-----------  -------------------  -------  -----  -----  ---------  ------------  ------  -------  ---------------  ----------
  Ethernet0  2304,2305,2306,2307     100G   9100    N/A  Ethernet0  PortChannel1    down     down  QSFP28 or later         N/A
  Ethernet1  2308,2309,2310,2311     100G   9100    N/A  Ethernet1  PortChannel1      up       up  QSFP28 or later         N/A
cisco@sonic:~$ 

2022-09-01 18:03:28,060: %UNICON-INFO: +++ sonic with via 'cli': executing command 'show interfaces status PortChannel1' +++
show

> The port channel should come back to **up** state when the member link is unshut.

In [30]:
# Bring up the member link Ethernet0
out = nodes['S0'].execute('sudo config interface startup Ethernet0')

# Verify that port channel is up when the member link is Up again
out = nodes['S0'].execute('show interface status Ethernet0-1')   
out = nodes['S0'].execute('show interfaces status PortChannel1')


2022-09-01 18:03:35,916: %UNICON-INFO: +++ sonic with via 'cli': executing command 'sudo config interface startup Ethernet0' +++
sudo config interface startup Ethernet0
cisco@sonic:~$ 

2022-09-01 18:03:36,498: %UNICON-INFO: +++ sonic with via 'cli': executing command 'show interface status Ethernet0-1' +++
show interface status Ethernet0-1
  Interface                Lanes    Speed    MTU    FEC      Alias          Vlan    Oper    Admin             Type    Asym PFC
-----------  -------------------  -------  -----  -----  ---------  ------------  ------  -------  ---------------  ----------
  Ethernet0  2304,2305,2306,2307     100G   9100    N/A  Ethernet0  PortChannel1    down       up  QSFP28 or later         N/A
  Ethernet1  2308,2309,2310,2311     100G   9100    N/A  Ethernet1  PortChannel1      up       up  QSFP28 or later         N/A
cisco@sonic:~$ 

2022-09-01 18:03:37,266: %UNICON-INFO: +++ sonic with via 'cli': executing command 'show interfaces status PortChannel1' +++
show i

#### Set up Port Channels with Fallback Option
> The fallback option allows an active member link of the port channel to establish the Link Aggregation (LAG) or port channel before receiving Link Aggregation Control Protocol (LACP) Protocol Data Unit (PDU) from its peer. LACP fallback will run on those member-links that have the lower port number. It runs on only one member port. This is an optional configuration. By default, the fallback option is set to **false**
> In the below cell, we will configure PortChannel1 of SPINE0 router with fallback option.

> First, shutdown Portchannel1 on LEAF0 so that it does not send any LACP PDUs to PortChannel1 on SPINE0. So the PortChannel1 on SPINE0 will be **down**.

In [10]:
# Shutting down the port channel on L0
out = nodes['L0'].execute('sudo config interface shutdown PortChannel1')
out = nodes['L0'].execute('show interfaces status PortChannel1')

# The portchannel will be down on SPINE0
out = nodes['S0'].execute('show interfaces status PortChannel1')



2022-09-01 17:48:44,301: %UNICON-INFO: +++ sonic with via 'cli': executing command 'sudo config interface shutdown PortChannel1' +++
sudo config interface shutdown PortChannel1
cisco@sonic:~$ 

2022-09-01 17:48:44,955: %UNICON-INFO: +++ sonic with via 'cli': executing command 'show interfaces status PortChannel1' +++
show interfaces status PortChannel1
   Interface    Lanes    Speed    MTU    FEC    Alias    Vlan    Oper    Admin    Type    Asym PFC
------------  -------  -------  -----  -----  -------  ------  ------  -------  ------  ----------
PortChannel1      N/A     200G   9100    N/A      N/A  routed    down     down     N/A         N/A
cisco@sonic:~$ 

2022-09-01 17:48:45,942: %UNICON-INFO: +++ sonic with via 'cli': executing command 'show interfaces status PortChannel1' +++
show interfaces status PortChannel1
   Interface    Lanes    Speed    MTU    FEC    Alias    Vlan    Oper    Admin    Type    Asym PFC
------------  -------  -------  -----  -----  -------  ------  ------ 

> Delete the existing PortChannel1 configurations on SPINE0 and then reconfigure the PortChannel1 on SPINE0 with fallback option set to **true**.

> You will see that the PortChannel1 on SPINE0 is **Up** even though it does not receive any LACP PDUs from PortChannel1 on LEAF0 router. This confirms that fallback option works.

In [12]:
# Deleting the existing port channel configurations 
out = nodes['S0'].execute('sudo config portchannel member del PortChannel1 Ethernet0')
out = nodes['S0'].execute('sudo config portchannel member del PortChannel1 Ethernet1')
out = nodes['S0'].execute('sudo config portchannel del PortChannel1')

# Configure the port channel with fallback option set to true
out = nodes['S0'].execute('sudo config portchannel add PortChannel1 --fallback=true')
out = nodes['S0'].execute('sudo config portchannel member add PortChannel1 Ethernet0')
out = nodes['S0'].execute('sudo config portchannel member add PortChannel1 Ethernet1')

# Verify the port channel on SPINE0 is up 
out = nodes['S0'].execute('show interface status Ethernet0-1')   
out = nodes['S0'].execute('show interfaces status PortChannel1')


2022-09-01 17:49:33,353: %UNICON-INFO: +++ sonic with via 'cli': executing command 'sudo config portchannel member del PortChannel1 Ethernet0' +++
sudo config portchannel member del PortChannel1 Ethernet0
cisco@sonic:~$ 

2022-09-01 17:49:33,961: %UNICON-INFO: +++ sonic with via 'cli': executing command 'sudo config portchannel member del PortChannel1 Ethernet1' +++
sudo config portchannel member del PortChannel1 Ethernet1
cisco@sonic:~$ 

2022-09-01 17:49:34,620: %UNICON-INFO: +++ sonic with via 'cli': executing command 'sudo config portchannel del PortChannel1' +++
sudo config portchannel del PortChannel1
cisco@sonic:~$ 

2022-09-01 17:49:35,238: %UNICON-INFO: +++ sonic with via 'cli': executing command 'sudo config portchannel add PortChannel1 --fallback=true' +++
sudo config portchannel add PortChannel1 --fallback=true
cisco@sonic:~$ 

2022-09-01 17:49:36,002: %UNICON-INFO: +++ sonic with via 'cli': executing command 'sudo config portchannel member add PortChannel1 Ethernet0' +++


> Finally unshut Portchannel1 on LEAF0. 

In [13]:
# Unshut the port channel on L0
out = nodes['L0'].execute('sudo config interface startup PortChannel1')
out = nodes['L0'].execute('show interfaces status PortChannel1')
out = nodes['S0'].execute('show interfaces status PortChannel1')


2022-09-01 17:49:43,835: %UNICON-INFO: +++ sonic with via 'cli': executing command 'sudo config interface startup PortChannel1' +++
sudo config interface startup PortChannel1
cisco@sonic:~$ 

2022-09-01 17:49:44,499: %UNICON-INFO: +++ sonic with via 'cli': executing command 'show interfaces status PortChannel1' +++
show interfaces status PortChannel1
   Interface    Lanes    Speed    MTU    FEC    Alias    Vlan    Oper    Admin    Type    Asym PFC
------------  -------  -------  -----  -----  -------  ------  ------  -------  ------  ----------
PortChannel1      N/A     200G   9100    N/A      N/A  routed      up       up     N/A         N/A
cisco@sonic:~$ 

2022-09-01 17:49:45,432: %UNICON-INFO: +++ sonic with via 'cli': executing command 'show interfaces status PortChannel1' +++
show interfaces status PortChannel1
   Interface    Lanes    Speed    MTU    FEC    Alias    Vlan    Oper    Admin    Type    Asym PFC
------------  -------  -------  -----  -----  -------  ------  ------  -

### Assign IP Addresses
> This step assigns IP addresses as per the topology diagram. And then saves the configurations.

In [31]:
out = nodes['S0'].execute('sudo config interface ip add PortChannel1 10.0.1.1/24')
out = nodes['S0'].execute('sudo config interface ip add PortChannel2 10.0.2.1/24')
out = nodes['S0'].execute('sudo config interface ip add Loopback0 10.10.10.100/32')
out = nodes['S0'].execute('sudo config save -y')
out = nodes['S1'].execute('sudo config interface ip add PortChannel1 10.0.3.1/24')
out = nodes['S1'].execute('sudo config interface ip add PortChannel2 10.0.4.1/24')
out = nodes['S1'].execute('sudo config interface ip add Loopback0 10.10.11.100/32')
out = nodes['S1'].execute('sudo config save -y')
out = nodes['L0'].execute('sudo config interface ip add PortChannel1 10.0.1.2/24')
out = nodes['L0'].execute('sudo config interface ip add PortChannel2 10.0.3.2/24')
out = nodes['L0'].execute('sudo config interface ip add Ethernet2 10.0.5.1/24')
out = nodes['L0'].execute('sudo config interface ip add Loopback0 10.10.10.200/32')
out = nodes['L0'].execute('sudo config save -y')
out = nodes['L1'].execute('sudo config interface ip add PortChannel2 10.0.2.2/24')
out = nodes['L1'].execute('sudo config interface ip add PortChannel1 10.0.4.2/24')
out = nodes['L1'].execute('sudo config interface ip add Ethernet2 10.0.6.1/24')
out = nodes['L1'].execute('sudo config interface ip add Loopback0 10.10.11.200/32')
out = nodes['L1'].execute('sudo config save -y')
out = nodes['trex'].execute('ifconfig eth1 10.0.5.2 netmask 255.255.255.0 up')
out = nodes['trex'].execute('ifconfig eth2 10.0.6.2 netmask 255.255.255.0 up')


2022-09-01 18:03:51,698: %UNICON-INFO: +++ sonic with via 'cli': executing command 'sudo config interface ip add PortChannel1 10.0.1.1/24' +++
sudo config interface ip add PortChannel1 10.0.1.1/24
cisco@sonic:~$ 

2022-09-01 18:03:52,256: %UNICON-INFO: +++ sonic with via 'cli': executing command 'sudo config interface ip add PortChannel2 10.0.2.1/24' +++
sudo config interface ip add PortChannel2 10.0.2.1/24
cisco@sonic:~$ 

2022-09-01 18:03:52,818: %UNICON-INFO: +++ sonic with via 'cli': executing command 'sudo config interface ip add Loopback0 10.10.10.100/32' +++
sudo config interface ip add Loopback0 10.10.10.100/32
cisco@sonic:~$ 

2022-09-01 18:03:53,465: %UNICON-INFO: +++ sonic with via 'cli': executing command 'sudo config save -y' +++
sudo config save -y
Running command: /usr/local/bin/sonic-cfggen -d --print-data > /etc/sonic/config_db.json
cisco@sonic:~$ 

2022-09-01 18:03:54,247: %UNICON-INFO: +++ sonic with via 'cli': executing command 'sudo config interface ip add PortCha

> Verify the configured IP addresses using the ```show ip interfaces``` command on the routers and ```ifconfig -a``` command on the TREX server.

In [32]:
for n in nodes:
   if (n == 'trex'):
      out = nodes[n].execute('ifconfig -a eth1')
      out = nodes[n].execute('ifconfig -a eth2')
   else:
      out = nodes[n].execute('show ip interfaces') 


2022-09-01 18:04:07,675: %UNICON-INFO: +++ sonic with via 'cli': executing command 'show ip interfaces' +++
show ip interfaces
Interface     Master    IPv4 address/mask    Admin/Oper    BGP Neighbor    Neighbor IP
------------  --------  -------------------  ------------  --------------  -------------
Loopback0               10.10.10.100/32      up/up         N/A             N/A
PortChannel1            10.0.1.1/24          up/up         N/A             N/A
PortChannel2            10.0.2.1/24          up/up         N/A             N/A
docker0                 240.127.1.1/24       up/down       N/A             N/A
eth0                    192.168.122.41/24    up/up         N/A             N/A
lo                      127.0.0.1/16         up/up         N/A             N/A
cisco@sonic:~$ 

2022-09-01 18:04:08,734: %UNICON-INFO: +++ sonic with via 'cli': executing command 'show ip interfaces' +++
show ip interfaces
Interface     Master    IPv4 address/mask    Admin/Oper    BGP Neighbor    Nei

### Configure eBGP

> The following section shows the eBGP (exterior Border Gateway Protocol) configuration on all spine and leaf routers. The spines are in Autonomous System (AS) 100 and the leaves in AS 200. 
> Note the usage of the ```allowas-in``` option in the ```neighbor``` command. The default behaviour of BGP is to reject learning routes from own AS. The ```allowas-in``` option overrides this default behavior and the nodes learn routes from own AS. 

In [33]:
out = nodes['S0'].execute ('''vtysh \
-c 'configure terminal' \
-c 'hostname SPINE_0' \
-c 'router-id 10.10.10.100' \
-c 'router bgp 100' \
-c 'no bgp ebgp-requires-policy' \
-c 'neighbor 10.0.2.2 remote-as 200' \
-c 'neighbor 10.0.1.2 remote-as 200' \
-c 'address-family ipv4 unicast' \
-c 'neighbor 10.0.2.2 allowas-in' \
-c 'neighbor 10.0.1.2 allowas-in' \
-c 'network 10.0.1.0/24' \
-c 'network 10.0.2.0/24' \
-c 'network 10.10.10.100/32' \
-c 'redistribute connected'
''')
out = nodes['S1'].execute ('''vtysh \
-c 'configure terminal' \
-c 'hostname SPINE_1' \
-c 'router-id 10.10.11.100' \
-c 'router bgp 100' \
-c 'no bgp ebgp-requires-policy' \
-c 'neighbor 10.0.3.2 remote-as 200' \
-c 'neighbor 10.0.4.2 remote-as 200' \
-c 'address-family ipv4 unicast' \
-c 'neighbor 10.0.3.2 allowas-in' \
-c 'neighbor 10.0.4.2 allowas-in' \
-c 'network 10.0.3.0/24' \
-c 'network 10.0.4.0/24' \
-c 'network 10.10.11.100/32' \
-c 'redistribute connected'
''')
out = nodes['L0'].execute ('''vtysh \
-c 'configure terminal' \
-c 'hostname LEAF_0' \
-c 'router-id 10.10.10.200' \
-c 'router bgp 200' \
-c 'no bgp ebgp-requires-policy' \
-c 'neighbor 10.0.1.1 remote-as 100' \
-c 'neighbor 10.0.3.1 remote-as 100' \
-c 'address-family ipv4 unicast' \
-c 'neighbor 10.0.1.1 allowas-in' \
-c 'neighbor 10.0.3.1 allowas-in' \
-c 'network 10.0.1.0/24' \
-c 'network 10.0.3.0/24' \
-c 'network 10.0.5.0/24' \
-c 'network 10.10.10.200/32' \
-c 'redistribute connected'
''')
out = nodes['L1'].execute ('''vtysh \
-c 'configure terminal' \
-c 'hostname LEAF_1' \
-c 'router-id 10.10.11.200' \
-c 'router bgp 200' \
-c 'no bgp ebgp-requires-policy' \
-c 'neighbor 10.0.2.1 remote-as 100' \
-c 'neighbor 10.0.4.1 remote-as 100' \
-c 'address-family ipv4 unicast' \
-c 'neighbor 10.0.2.1 allowas-in' \
-c 'neighbor 10.0.4.1 allowas-in' \
-c 'network 10.0.2.0/24' \
-c 'network 10.0.4.0/24' \
-c 'network 10.0.6.0/24' \
-c 'network 10.10.11.200/32' \
-c 'redistribute connected'
''')


2022-09-01 18:04:19,955: %UNICON-INFO: +++ sonic with via 'cli': executing command 'vtysh -c 'configure terminal' -c 'hostname SPINE_0' -c 'router-id 10.10.10.100' -c 'router bgp 100' -c 'no bgp ebgp-requires-policy' -c 'neighbor 10.0.2.2 remote-as 200' -c 'neighbor 10.0.1.2 remote-as 200' -c 'address-family ipv4 unicast' -c 'neighbor 10.0.2.2 allowas-in' -c 'neighbor 10.0.1.2 allowas-in' -c 'network 10.0.1.0/24' -c 'network 10.0.2.0/24' -c 'network 10.10.10.100/32' -c 'redistribute connected'' +++
' -c 'redistribute connected'ress-family ipv4 unicast' -c 'neighbor 10.0.2.2 allowas-in' -c 'neighbor 10.0.1.2 allowas-in' -c 'network 10.0.1.0/24' -c 'network 10.0.2.0/24' -c 'network 10.10.10.100/32'
cisco@sonic:~$ 

2022-09-01 18:04:20,372: %UNICON-INFO: +++ sonic with via 'cli': executing command 'vtysh -c 'configure terminal' -c 'hostname SPINE_1' -c 'router-id 10.10.11.100' -c 'router bgp 100' -c 'no bgp ebgp-requires-policy' -c 'neighbor 10.0.3.2 remote-as 200' -c 'neighbor 10.0.4.2 

> Execute the command ```show ip bgp``` on all tier-0 and tier-1 routers to ensure eBGP is up.

In [34]:
for n in nodes:
   if (n != 'trex'):
      out = nodes[n].execute('''vtysh \
      -c 'show ip bgp'
      ''')


2022-09-01 18:04:24,941: %UNICON-INFO: +++ sonic with via 'cli': executing command 'vtysh       -c 'show ip bgp'' +++
vtysh       -c 'show ip bgp'
BGP table version is 38, local router ID is 10.10.10.100, vrf id 0
Default local pref 100, local AS 100
Status codes:  s suppressed, d damped, h history, * valid, > best, = multipath,
               i internal, r RIB-failure, S Stale, R Removed
Nexthop codes: @NNN nexthop's vrf id, < announce-nh-self
Origin codes:  i - IGP, e - EGP, ? - incomplete

   Network          Next Hop            Metric LocPrf Weight Path
*  10.0.1.0/24      10.0.2.2                               0 200 100 i
*                   10.0.1.2                 0             0 200 i
*                   0.0.0.0                  0         32768 ?
*>                  0.0.0.0                  0         32768 i
*  10.0.2.0/24      10.0.2.2                 0             0 200 i
*                   10.0.1.2                               0 200 100 i
*                   0.0.0.0      

> This section checks the routes learnt through BGP by executing the command ```show ip route``` on all nodes in tier-0 and tier-1. The lines starting with the letter **B** are the routes learnt through BGP.

In [35]:
for n in nodes:
   if (n != 'trex'):
      out = nodes[n].execute('''vtysh \
      -c 'show ip route'
      ''')


2022-09-01 18:04:29,779: %UNICON-INFO: +++ sonic with via 'cli': executing command 'vtysh       -c 'show ip route'' +++
vtysh       -c 'show ip route'
Codes: K - kernel route, C - connected, S - static, R - RIP,
       O - OSPF, I - IS-IS, B - BGP, E - EIGRP, N - NHRP,
       T - Table, v - VNC, V - VNC-Direct, A - Babel, D - SHARP,
       F - PBR, f - OpenFabric,
       > - selected route, * - FIB route, q - queued, r - rejected, b - backup

K>* 0.0.0.0/0 [0/202] via 192.168.122.1, eth0, 00:35:11
C>* 10.0.1.0/24 is directly connected, PortChannel1, 00:00:38
C>* 10.0.2.0/24 is directly connected, PortChannel2, 00:00:38
B>* 10.0.3.0/24 [20/0] via 10.0.1.2, PortChannel1, weight 1, 00:00:31
B>* 10.0.4.0/24 [20/0] via 10.0.2.2, PortChannel2, weight 1, 00:00:27
B>* 10.0.5.0/24 [20/0] via 10.0.1.2, PortChannel1, weight 1, 00:00:31
B>* 10.0.6.0/24 [20/0] via 10.0.2.2, PortChannel2, weight 1, 00:00:27
C>* 10.10.10.100/32 is directly connected, Loopback0, 00:14:33
B>* 10.10.10.200/32 [20/0] vi

> Ping IP addresses on the LEAF0 (L0) router from the LEAF1 (L1) router and vice-versa. This ensures that the 3-stage Clos network is ready for end-to-end traffic flow.

In [36]:
out = nodes['L1'].execute('ping -c5 10.0.5.1')
out = nodes['L1'].execute('ping -c5 10.10.10.200')
out = nodes['L0'].execute('ping -c5 10.0.6.1')
out = nodes['L0'].execute('ping -c5 10.10.11.200')


2022-09-01 18:04:34,247: %UNICON-INFO: +++ sonic with via 'cli': executing command 'ping -c5 10.0.5.1' +++
ping -c5 10.0.5.1
PING 10.0.5.1 (10.0.5.1) 56(84) bytes of data.
64 bytes from 10.0.5.1: icmp_seq=1 ttl=63 time=14.6 ms
64 bytes from 10.0.5.1: icmp_seq=2 ttl=63 time=12.5 ms
64 bytes from 10.0.5.1: icmp_seq=3 ttl=63 time=17.9 ms
64 bytes from 10.0.5.1: icmp_seq=4 ttl=63 time=13.4 ms
64 bytes from 10.0.5.1: icmp_seq=5 ttl=63 time=13.7 ms

--- 10.0.5.1 ping statistics ---
5 packets transmitted, 5 received, 0% packet loss, time 10ms
rtt min/avg/max/mdev = 12.484/14.409/17.897/1.871 ms
cisco@sonic:~$ 

2022-09-01 18:04:38,431: %UNICON-INFO: +++ sonic with via 'cli': executing command 'ping -c5 10.10.10.200' +++
ping -c5 10.10.10.200
PING 10.10.10.200 (10.10.10.200) 56(84) bytes of data.
64 bytes from 10.10.10.200: icmp_seq=1 ttl=63 time=30.2 ms
64 bytes from 10.10.10.200: icmp_seq=2 ttl=63 time=15.0 ms
64 bytes from 10.10.10.200: icmp_seq=3 ttl=63 time=21.3 ms
64 bytes from 10.10.10

**Pinging to remote leaf ip address is successful and so the 3-stage Clos network is up and ready for data traffic**

### Send Traffic from TREX

> [TREX](https://trex-tgn.cisco.com/trex/doc/trex_manual.html#_introduction) is a software traffic generator that runs on Linux. To simulate server-to-server traffic flow across a 3-stage Clos network, connect the TREX software traffic generator ports as the end-hosts to LEAF0 and LEAF1. 

> Details of traffic stream injected into LEAF0 from TREX:
> * Source IP address: 10.0.5.2
> * Destination IP address: 10.0.6.2

> Details of traffic stream injected into LEAF1 from TREX:
> * Source IP address: 10.0.6.2
> * Destination IP address: 10.0.5.2

<center><img src="./images/portchannel-traffic.png" width="700"/></center>


> The function ```generate_bidir_traffic``` generates a bidirectional traffic burst for 1 second. After the cell execution, check ```Total-tx-pkt``` and ```Total-rx-pkt``` in the ```summary stats``` at the end of the output to ensure that there is no traffic loss.

In [37]:
trex_ipaddress = str(nodes['trex'].connections.cli.ip)
trex_port = str(nodes['trex'].connections.cli.port)

generate_bidir_traffic(trex_ipaddress, trex_port)

Last login: Thu Sep  1 17:50:57 2022 from sonic-6
[root@localhost ~]# cd /opt/cisco/trex/latest/cap2/; ls -lart test-new.yaml; cd  /opt/cisco/trex/latest/
-rw-r--r-- 1 root root 556 Sep  1 18:04 test-new.yaml
[root@localhost latest]# ./t-rex-64 -f cap2/test-new.yaml -m 300 -d 1
exit
Trying to bind to igb_uio ...
/usr/bin/python3 dpdk_nic_bind.py --bind=igb_uio 0000:00:04.0 0000:00:05.0 
The ports are bound/configured.
Starting  TRex v2.99 please wait  ... 
 set driver name net_virtio 
 driver capability  : SLRO 
 set dpdk queues mode to ONE_QUE 
 Number of ports found: 2
zmq publisher at: tcp://*:4500
 wait 1 sec .
port : 0 
------------
link         :  link : Link Up - speed 100000 Mbps - full-duplex
promiscuous  : 0 
port : 1 
------------
link         :  link : Link Up - speed 100000 Mbps - full-duplex
promiscuous  : 0 
 number of ports         : 2 
 max cores for 2 ports   : 1 
 tx queues per port      : 3 
no client generator pool configured, using default pool
no server generator

### Verify Traffic Statistics

> Check the interface counters on the nodes to ensure that the traffic is sent and received as seen in the traffic generator output.
    
> Note: "show interface counters rif" displaying 0s for all fields, is a known issue on the emulator. Users will experience expected behaviours on the physical setup.


In [39]:
for n in nodes:
   if (n != 'trex'):
      out = nodes[n].execute('show interface counters rif')


2022-09-01 18:05:56,727: %UNICON-INFO: +++ sonic with via 'cli': executing command 'show interface counters rif' +++
show interface counters rif
       IFACE    RX_OK    RX_BPS    RX_PPS    RX_ERR    TX_OK    TX_BPS    TX_PPS    TX_ERR
------------  -------  --------  --------  --------  -------  --------  --------  --------
PortChannel1       38  0.00 B/s    0.00/s         0        5  0.00 B/s    0.00/s         0
PortChannel2       36  0.00 B/s    0.00/s         0       10  0.00 B/s    0.00/s         0
cisco@sonic:~$ 

2022-09-01 18:05:57,726: %UNICON-INFO: +++ sonic with via 'cli': executing command 'show interface counters rif' +++
show interface counters rif
       IFACE    RX_OK    RX_BPS    RX_PPS    RX_ERR    TX_OK    TX_BPS    TX_PPS    TX_ERR
------------  -------  --------  --------  --------  -------  --------  --------  --------
PortChannel1       31  0.00 B/s    0.00/s         0       15  0.00 B/s    0.00/s         0
PortChannel2       48  0.00 B/s    0.00/s         0    

> You have now successfully brought up a simple 3-stage Clos network, sent traffic across it. This forms the IP-BGP underlay for your data center network. 

> Scale up these configurations as per the number of devices in your data centre. 

### Clean Up Emulator Session

> Delete the port channels

In [40]:
out = nodes['S0'].execute('sudo config portchannel member del PortChannel1 Ethernet0')
out = nodes['S0'].execute('sudo config portchannel member del PortChannel1 Ethernet1')
out = nodes['S0'].execute('sudo config portchannel del PortChannel1')
out = nodes['S0'].execute('sudo config portchannel member del PortChannel2 Ethernet2')
out = nodes['S0'].execute('sudo config portchannel member del PortChannel2 Ethernet3')
out = nodes['S0'].execute('sudo config portchannel del PortChannel2')
out = nodes['S1'].execute('sudo config portchannel member del PortChannel1 Ethernet0')
out = nodes['S1'].execute('sudo config portchannel member del PortChannel1 Ethernet1')
out = nodes['S1'].execute('sudo config portchannel del PortChannel1')
out = nodes['S1'].execute('sudo config portchannel member del PortChannel2 Ethernet2')
out = nodes['S1'].execute('sudo config portchannel member del PortChannel2 Ethernet3')
out = nodes['S1'].execute('sudo config portchannel del PortChannel2')
out = nodes['L0'].execute('sudo config portchannel member del PortChannel1 Ethernet0')
out = nodes['L0'].execute('sudo config portchannel member del PortChannel1 Ethernet1')
out = nodes['L0'].execute('sudo config portchannel del PortChannel1')
out = nodes['L0'].execute('sudo config portchannel member del PortChannel2 Ethernet3')
out = nodes['L0'].execute('sudo config portchannel member del PortChannel2 Ethernet4')
out = nodes['L0'].execute('sudo config portchannel del PortChannel2')
out = nodes['L1'].execute('sudo config portchannel member del PortChannel1 Ethernet0')
out = nodes['L1'].execute('sudo config portchannel member del PortChannel1 Ethernet1')
out = nodes['L1'].execute('sudo config portchannel del PortChannel1')
out = nodes['L1'].execute('sudo config portchannel member del PortChannel2 Ethernet3')
out = nodes['L1'].execute('sudo config portchannel member del PortChannel2 Ethernet4')
out = nodes['L1'].execute('sudo config portchannel del PortChannel2')


2022-09-01 18:06:19,682: %UNICON-INFO: +++ sonic with via 'cli': executing command 'sudo config portchannel member del PortChannel1 Ethernet0' +++
sudo config portchannel member del PortChannel1 Ethernet0
cisco@sonic:~$ 

2022-09-01 18:06:20,356: %UNICON-INFO: +++ sonic with via 'cli': executing command 'sudo config portchannel member del PortChannel1 Ethernet1' +++
sudo config portchannel member del PortChannel1 Ethernet1
cisco@sonic:~$ 

2022-09-01 18:06:20,888: %UNICON-INFO: +++ sonic with via 'cli': executing command 'sudo config portchannel del PortChannel1' +++
sudo config portchannel del PortChannel1
cisco@sonic:~$ 

2022-09-01 18:06:21,442: %UNICON-INFO: +++ sonic with via 'cli': executing command 'sudo config portchannel member del PortChannel2 Ethernet2' +++
sudo config portchannel member del PortChannel2 Ethernet2
cisco@sonic:~$ 

2022-09-01 18:06:22,001: %UNICON-INFO: +++ sonic with via 'cli': executing command 'sudo config portchannel member del PortChannel2 Ethernet3' ++

> Remove the router configurations so that you can play other notebooks

In [41]:
# Remove the Router configs and release the interfaces used by trex
for n in nodes:
   if (n != 'trex'):
      print("cfg cleaned")
      out = nodes[n].execute('sudo rm /etc/sonic/config_db.json')
   else:
      out = nodes[n].execute('cd /opt/cisco/trex/latest/')
      out = nodes['trex'].execute('sudo ./dpdk_nic_bind.py --force -u 00:04.0')
      out = nodes['trex'].execute('sudo ./dpdk_nic_bind.py --force -u 00:05.0')
      out = nodes['trex'].execute('sudo ./dpdk_nic_bind.py --bind=virtio-pci 00:04.0')
      out = nodes['trex'].execute('sudo ./dpdk_nic_bind.py --bind=virtio-pci 00:05.0')

cfg cleaned

2022-09-01 18:06:39,224: %UNICON-INFO: +++ sonic with via 'cli': executing command 'sudo rm /etc/sonic/config_db.json' +++
sudo rm /etc/sonic/config_db.json
cisco@sonic:~$ 
cfg cleaned

2022-09-01 18:06:39,387: %UNICON-INFO: +++ sonic with via 'cli': executing command 'sudo rm /etc/sonic/config_db.json' +++
sudo rm /etc/sonic/config_db.json
cisco@sonic:~$ 
cfg cleaned

2022-09-01 18:06:39,549: %UNICON-INFO: +++ sonic with via 'cli': executing command 'sudo rm /etc/sonic/config_db.json' +++
sudo rm /etc/sonic/config_db.json
cisco@sonic:~$ 
cfg cleaned

2022-09-01 18:06:39,710: %UNICON-INFO: +++ sonic with via 'cli': executing command 'sudo rm /etc/sonic/config_db.json' +++
sudo rm /etc/sonic/config_db.json
cisco@sonic:~$ 

2022-09-01 18:06:39,872: %UNICON-INFO: +++ localhost with via 'cli': executing command 'cd /opt/cisco/trex/latest/' +++
cd /opt/cisco/trex/latest/
[root@localhost latest]# 

2022-09-01 18:06:40,048: %UNICON-INFO: +++ localhost with via 'cli': executing co

> Do let us know of your feedback or queries about this notebook at mig-notebooks@cisco.com.